In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# ! ! python /mnt/cube/tsainbur/Projects/github_repos/childes_mi_project/childes_mi/information_theory/emi/setup.py uild_ext --inplace
from childes_mi.information_theory.emi.nij_op_cython import nij_op_cython

ModuleNotFoundError: No module named 'childes_mi.information_theory.emi.nij_op_cython'

In [3]:
#! python /mnt/cube/tsainbur/Projects/github_repos/childes_mi_project/childes_mi/information_theory/emi/setup.py uild_ext --inplace

In [4]:
from scipy.special import gammaln
import numpy as np
from tqdm.autonotebook import tqdm
from joblib import Parallel, delayed

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [5]:
# ModuleNotFoundError: No module named '_expected_mutual_info_fast'

In [6]:
from childes_mi.information_theory.emi._nij_op_cython import nij_op_cython


In [7]:
from childes_mi.information_theory.emi._expected_mutual_info_fast import expected_mutual_information
expected_mutual_information

ModuleNotFoundError: No module named 'childes_mi.information_theory.emi._expected_mutual_info_fast'

In [23]:
from sklearn.metrics.cluster._expected_mutual_info_fast import expected_mutual_information
expected_mutual_information

<function sklearn.metrics.cluster._expected_mutual_info_fast.expected_mutual_information>

In [27]:
from sklearn.metrics import cluster

In [28]:
??cluster

In [9]:
import numpy as np
import scipy.sparse
from sklearn.metrics.cluster import contingency_matrix
from sklearn.metrics.cluster import adjusted_mutual_info_score

In [10]:
seed = 42
k = 10
n_samples = 10000
random_labels = np.random.RandomState(seed).randint
distribution_a = random_labels(low=0, high=k, size=n_samples)
distribution_b = random_labels(low=0, high=k, size=n_samples)
distribution_b[distribution_b > 5] = 5
C = contingency_matrix(distribution_a, distribution_b, sparse=True)
n_samples = C.sum()
C.shape, n_samples

((10, 6), 10000)

In [11]:
def emi_parallel(contingency, n_samples, use_cython = False, n_jobs=-1):
    """
    EMI without pregenerating lookup table for reduced memory
    https://github.com/clajusch/ClEvaR/blob/master/R/Calculations.R
    """
    
    print("EMI reduced memory parallel")
    s1 = np.array(np.sum(contingency, axis=1, dtype="int").flatten()).flatten()
    s2 = np.array(np.sum(contingency, axis=0, dtype="int").flatten()).flatten()
    N = n_samples
    l1 = len(s1)
    l2 = len(s2)

    nijs = np.arange(0, max(np.max(s1), np.max(s2)) + 1, dtype="float")
    nijs[0] = 1
    term1 = nijs / N

    gln_a = gammaln(s1 + 1)
    gln_b = gammaln(s2 + 1)
    gln_Na = gammaln(N - s1 + 1)
    gln_Nb = gammaln(N - s2 + 1)
    gln_N = gammaln(N + 1)
    gln_nij = gammaln(nijs + 1)
    
    if use_cython:
        nij_func = nij_op_cython
    else:
        nij_func = nij_op

    with Parallel(n_jobs=n_jobs, verbose=0, prefer=None) as parallel:
    
        emi = parallel(
            delayed(nij_func)(s1[i], s2, l2, N, term1, nijs, i, gln_a, gln_b, gln_Na, gln_Nb, gln_N, gln_nij)
            for i in tqdm(range(l1), desc="compute emi", mininterval=0.25)
        )
            

    return np.sum(emi)

In [12]:
def nij_op(s1i, s2, l2, N, term1, nijs, i, gln_a, gln_b, gln_Na, gln_Nb, gln_N, gln_nij):
    emif = 0
    for j in range(l2):
        s2j = s2[j]
        min_nij = np.max([1, s1i - N + s2j])
        max_nij = np.min([s1i, s2j]) + 1
        nij = np.arange(min_nij, max_nij)
        t1 = term1[nij]
        t2 = np.log(N * nijs[nij]) - np.log(s1i * s2j)

        gln = (
            gln_a[i] + 
            gln_b[j] + 
            gln_Na[i] + 
            gln_Nb[j] - 
            gln_N - 
            gln_nij[nij] - 
            gammaln(s1i - nij + 1) - 
            gammaln(s2j - nij + 1) -
            gammaln(N - s1i- s2j + nij + 1)
        )

        t3 = np.exp(gln)
        emi = sum(t1 * t2 * t3)
        emif += emi
    return emif

In [13]:
# 0.002254254559813846
emi_parallel(C, n_samples)

EMI reduced memory parallel


0.002254254559813846

In [14]:
C = scipy.sparse.load_npz('test.npz')
n_samples = C.sum()
n_samples, C.shape

(14975, (5372, 5361))

In [15]:
# [01:19<00:00, 67.29it/s] 6.415867094292364
#emi_parallel(C, n_samples)

### test cython

In [16]:
%load_ext Cython

In [17]:
%%cython -a

from scipy.special import gammaln
import numpy as np
cimport numpy as np
cimport cython


np.import_array()
ctypedef np.float64_t DOUBLE

def nij_op_cython(int s1i, np.ndarray s2, int l2, int N, np.ndarray term1, 
                  np.ndarray nijs, int i, np.ndarray gln_a, np.ndarray gln_b, 
                  np.ndarray gln_Na, np.ndarray gln_Nb, float gln_N, np.ndarray gln_nij):
    cdef DOUBLE emi
    cdef np.ndarray[DOUBLE] gln, t1, t2, t3
    cdef np.ndarray[np.int32_t] nij 
    cdef int min_nij, max_nij, s2j
    emif = 0
    cdef Py_ssize_t j
    for j in range(l2):
        s2j = s2[j]
        min_nij = max([1, s1i + s2j - N])
        max_nij = min([s1i, s2j])
        nij = np.arange(min_nij, max_nij).astype(np.int32, copy=False) + 1
        t1 = term1[nij]
        t2 = np.log(N * nijs[nij]) - np.log(s1i * s2j)

        gln = (
            gln_a[i] + 
            gln_b[j] + 
            gln_Na[i] + 
            gln_Nb[j] - 
            gln_N - 
            gln_nij[nij] - 
            gammaln(s1i - nij + 1) - 
            gammaln(s2j - nij + 1) -
            gammaln(N - s1i- s2j + nij + 1)
        )

        t3 = np.exp(gln)
        emi = sum(t1 * t2 * t3)
        emif += emi
    return emif"""

'%%cython -a\n\nfrom scipy.special import gammaln\nimport numpy as np\ncimport numpy as np\ncimport cython\n\n\nnp.import_array()\nctypedef np.float64_t DOUBLE\n\ndef nij_op_cython(int s1i, np.ndarray s2, int l2, int N, np.ndarray term1, \n                  np.ndarray nijs, int i, np.ndarray gln_a, np.ndarray gln_b, \n                  np.ndarray gln_Na, np.ndarray gln_Nb, float gln_N, np.ndarray gln_nij):\n    cdef DOUBLE emi\n    cdef np.ndarray[DOUBLE] gln, t1, t2, t3\n    cdef np.ndarray[np.int32_t] nij \n    cdef int min_nij, max_nij, s2j\n    emif = 0\n    cdef Py_ssize_t j\n    for j in range(l2):\n        s2j = s2[j]\n        min_nij = max([1, s1i + s2j - N])\n        max_nij = min([s1i, s2j])\n        nij = np.arange(min_nij, max_nij).astype(np.int32, copy=False) + 1\n        t1 = term1[nij]\n        t2 = np.log(N * nijs[nij]) - np.log(s1i * s2j)\n\n        gln = (\n            gln_a[i] + \n            gln_b[j] + \n            gln_Na[i] + \n            gln_Nb[j] - \n          

In [18]:
seed = 42
k = 10
n_samples = 10000
random_labels = np.random.RandomState(seed).randint
distribution_a = random_labels(low=0, high=k, size=n_samples)
distribution_b = random_labels(low=0, high=k, size=n_samples)
distribution_b[distribution_b > 5] = 5
C = contingency_matrix(distribution_a, distribution_b, sparse=True)
n_samples = C.sum()
C.shape, n_samples

((10, 6), 10000)

In [19]:
emi_parallel(C, n_samples, use_cython = True, n_jobs=1)

EMI reduced memory parallel


0.002250086501340412

In [20]:
C = scipy.sparse.load_npz('test.npz')
n_samples = C.sum()
n_samples, C.shape

(14975, (5372, 5361))

In [29]:
emi_parallel(C, n_samples, use_cython = True, n_jobs=2)

EMI reduced memory parallel


BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.